In [164]:
from time import sleep
from json import dumps
from kafka import KafkaProducer
import pandas as pd
from elasticsearch import helpers, Elasticsearch
import elasticsearch


producer = KafkaProducer(bootstrap_servers=['35.232.117.118:9092'], 
                        value_serializer=lambda x: dumps(x).encode('utf-8'),
                        api_version = (0,10))
def getFields(es_index):
    es = Elasticsearch([{'host': '34.73.60.209', 'port': 9200}])
    ind_client = elasticsearch.client.IndicesClient(es)

    fields = list(ind_client.get_mapping()[es_index]['mappings']['type']['properties'].keys())
    return fields


def crimesUpload(path, producer, kafka_topic, es_index):

    df = pd.read_csv(path)
    fields = getFields(es_index)
    if sorted(fields) == sorted(list(df.columns)):
        print('Uploading to Kafka topic:', kafka_topic)
        df.apply(lambda x: producer.send(kafka_topic, value = x.to_dict()), axis=1)
    else:
        print('Error. The data fields of ' + str(path) + ' must match:', fields)
    
    return df
    
crimesUpload('sample_upload_bad.csv', producer, 'crimes', 'combined_crimes' )             



Error. The data fields of sample_upload_bad.csv must match: ['Case_ID', 'Crime_Description', 'Date', 'Geolocation', 'Location_Description', 'Neighborhood_Score', 'Predicted_Neighborhood_Score', 'Predicted_Rental_Price', 'Rental_Price', 'Zipcode']


,XXX,Date,Crime_Description,Location_Description,Geolocation,Zipcode,Rental_Price,Neighborhood_Score,Predicted_Rental_Price,Predicted_Neighborhood_Score
0,1,03/04/2019,Arson,House,"(33.9328, -118.2621)",90061,2090,25,NaN,NaN
1,2,03/05/2019,Murder,Store,"(33.9581, -118.2651)",90003,2047,50,NaN,NaN
2,3,03/05/2019,Homicide,Church,"(33.8773, -118.2867)",90248,2430,75,NaN,NaN
3,4,03/06/2019,Theft,House,"(33.9492, -118.2827)",90044,2116,80,NaN,NaN
